# [0] [Total3D pickles] layout_emitter + 3D lighting

In [18]:
with open('/home/ruizhu/Documents/Projects/Total3DUnderstanding/valid_scenes.pkl', 'rb') as f:
    valid_scenes = pickle.load(f)


In [22]:

valid_scenes.index('mainDiffMat_xml1/scene0385_01')

5934

In [1]:
# borrowed from Total3D repo -> notebooks/parse_OR_labels-lighting-reindexed%2Bassigned-V4.ipynb
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')

import os
import json

%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib widget

# dataset = 'sunrgbd'
dataset = 'OR'

mode = 'val'
# mode = 'train'
# mode = 'train-tmp'

# split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': 'utils_OR/openrooms/preprocessed-full'}
split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': '/home/ruizhu/Documents/Projects/semanticInverse/train/data/openrooms/list_OR_V4full/list'}

if dataset == 'sunrgbd':
    split_file = os.path.join(split_path_dict[dataset], mode + '.json')
    with open(split_file) as file:
        split = json.load(file)
elif dataset == 'OR':
    split_file = os.path.join(split_path_dict[dataset], mode + '.txt')
    with open(split_file) as f:
        mylist = f.read().splitlines() 
    split = [x.split(' ') for x in mylist]

print(len(split))

5664


In [121]:
import torch
import numpy as np
from pathlib import Path
from utils.utils_total3D.utils_OR_visualize import format_layout, Box
from utils.utils_total3D.data_config import OR4XCLASSES_dict, NYU40CLASSES
from utils.utils_total3D.utils_others import OR4X_mapping_catInt_to_RGB, OR4X_mapping_catStr_to_RGB
from utils.utils_total3D.utils_OR_vis_labels import read_vis_scene_3d, RGB_to_01
import pickle

OR = 'OR45'
classes = OR4XCLASSES_dict[OR] if dataset == 'OR' else NYU40CLASSES
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catInt_to_RGB['light']), 'rb') as f:
    OR_mapping_catInt_to_RGB = pickle.load(f)[OR]
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catStr_to_RGB['light']), 'rb') as f:
    OR_mapping_catStr_to_RGB = pickle.load(f)[OR]

import random
# image_id = random.randint(0, len(split))
# image_id = 3
# pickle_path_ori = split[image_id]

# pickle_id = random.randint(0, len(split))
# pickle_id = 10
# pickle_id = 26
# pickle_id = 27

# pickle_id = 16 # blue room, sofa, window
# wall_idx, cell_i, cell_j = 2, 3, 3

# pickle_id = 6
# wall_idx, cell_i, cell_j = 4, 2, 4

pickle_id = 18 # classroom
wall_idx, cell_i, cell_j = 4, 3, 3

# random
pickle_id = 1 # [!!! 0-based]

In [136]:

naming = split[pickle_id][2]
print(naming)

# naming = 'main_xml/scene0096_00/im_6.hdr'
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
# naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
# pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
meta_split = naming.split('/')[0]
scene_name = naming.split('/')[1]
withinsequence_id = int(naming.split('/')[2].split('.')[0].split('_')[1])
# assert pickle_id==withinsequence_id

train_test_data_path = '/data/ruizhu/OR-V4full-withMesh20210506-val500-OR45_total3D_train_test_data'

pickle_path_ori = '%s/%s/%s/%s'%(train_test_data_path, naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


grid_size = 8

pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

print(pickle_path)

# ======= read saved results from evaluation

# GT envmap input + overfit to val200
# results_folder = '20210503-170713-train_POD_emitterEst_lightAccuV3_bs2x4_GTsampledEnvMap-GTBRDFLight_overFitVal_fixedDataScaleCellOrder0502data'; tid=10000

# est
results_folder = '20210503-155348-train_POD_emitterEst_lightAccuV3_bs2x4_RuitrainedBRDF-light_fixedDataScaleCellOrder0502data'; tid=12000

layout_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_layout_info_tid%d-%d.pickle'%(results_folder, tid, 0)
emitter_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_cells_vis_info_list_tid%d-%d.pickle'%(results_folder, tid, 0)
results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%(results_folder, 0)

with open(layout_results_pickle_path, 'rb') as f:
    layout_pickle_dict = pickle.load(f)
    print(layout_pickle_dict.keys())
with open(emitter_results_pickle_path, 'rb') as f:
    emitter_pickle_dict = pickle.load(f)
    print(emitter_pickle_dict.keys())

cells_vis_info_list = emitter_pickle_dict['cells_vis_info_list']
cam_R_pred = layout_pickle_dict['pre_cam_R']
pred_layout = layout_pickle_dict['pre_layout']
emitter_cls_prob_PRED = emitter_pickle_dict['emitter_cls_prob_PRED']
cell_info_grid_PRED = emitter_pickle_dict['cell_info_grid_PRED']


with open(results_emitter_input_pickle_path, 'rb') as f:
    results_emitter_input_pickle_dict = pickle.load(f)

env_scale, hdr_scale = results_emitter_input_pickle_dict['env_scale'], results_emitter_input_pickle_dict['hdr_scale']
print(env_scale, hdr_scale)

import pickle
with open(pickle_path, 'rb') as f:
    sequence = pickle.load(f)
print(sequence.keys())

with open(pickle_emitters_path, 'rb') as f:
    sequence_emitters = pickle.load(f)
print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
with open(pickle_emitters_assign_info_list_path, 'rb') as f:
    sequence_emitter2wall_assign_info_dict = pickle.load(f)
print(sequence_emitter2wall_assign_info_dict.keys())

import pickle5 as pickle
emitter_representation_type = '0ambient'
emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
    emitters_prop_dict_representation_dict = pickle.load(f)

# transform_to_total3d_coords_dict = {'transform_R': frame.transform_R, 'transform_t': frame.transform_t}
transform_to_total3d_coords_dict_path = Path(train_test_data_path) / meta_split / scene_name / ('transform_to_total3d_coords_dict_%d.pkl'%withinsequence_id)
import pickle5 as pickle
with open(transform_to_total3d_coords_dict_path, 'rb') as f:
    transform_to_total3d_coords_dict = pickle.load(f)

# RAW_data_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layout_labels_V4full-detachEmitterRERERE20210420/'
# emitter_dict_pickle_file = Path(RAW_data_path) / meta_split / scene_name / ('emitter_dict_%s_%d.pickle'%('0ambient', withinsequence_id))
# with open(emitter_dict_pickle_file, "rb") as f:
#     emitter_dict_RAW = pickle.load(f)


%matplotlib widget

gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
gt_boxes_dict_RE = {key: [x[key] for x in sequence['boxes']['bdb3D_full']] for key in sequence['boxes']['bdb3D_full'][0]}
len_keys = [len(gt_boxes_dict_RE[key]) for key in gt_boxes_dict_RE]
assert len(list(set(len_keys))) == 1
gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
random_id_list = [x['random_id'] for x in sequence['boxes']['bdb2D_full']]

gt_obj_paths_list_normalized = [x.replace('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped', '/home/ruizhu/Documents/data/OR-sampledMeshes').replace('alignedNew.obj', 'alignedNew_normalized.obj') for x in gt_obj_paths_list]
assert len(gt_obj_paths_list_normalized)==len(gt_obj_paths_list)==len_keys[0]
gt_boxes_dict_RE['obj_path_normalized'] = gt_obj_paths_list_normalized
gt_boxes_dict_RE['random_id'] = random_id_list
gt_boxes_dict_RE['bdb2d'] = [[x['x1'], x['y1'], x['x2'], x['y2']] for x in sequence['boxes']['bdb2D_full']]
gt_boxes_dict_RE['mask'] = [x for x in sequence['boxes']['mask']]
assert len(gt_boxes_dict_RE['mask']) == len(gt_boxes_dict_RE['bdb2d'])
assert len(gt_boxes_dict_RE['mask']) == len(sequence['boxes']['if_valid'])
for bbox_idx in range(len(gt_boxes_dict_RE['if_valid'])):
    if gt_boxes_dict_RE['mask'][bbox_idx] is None:
        gt_boxes_dict_RE['if_valid'][bbox_idx] = False
#     if sequence['boxes']['bdb2D_full']['if_is_indoor_obj'] == False:
#         gt_boxes_dict_RE['if_valid'][bbox_idx] = False
#     else:

gt_layout = sequence['layout']['bdb3D']

from utils.utils_total3D.data_config import Dataset_Config
from utils.utils_total3D.utils_OR_layout import to_dict_tensor

from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

OR = 'OR45'
dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                  torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                  torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

# ======== emitters_list
emitters_obj_list = []
for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
    obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'random_id': sequence_emitters['boxes']['random_id'][x], \
                    'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                    'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                    'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
    obj_random_id = sequence_emitters['boxes']['random_id'][x]
    emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
    if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window' and 'light_center_world_total3d' in emitter_prop_total3d:
        light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
        light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
    else:
        light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
        light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
    obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]

    obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

    emitters_obj_list.append(obj_dict_new)

emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
cell_info_grid_GT = []
for wall_idxx in range(6):
    for i in range(grid_size):
        for j in range(grid_size):
            cell_info = cell_info_grid_GT_includeempty[wall_idxx * grid_size**2 + i * grid_size + j]
            if cell_info['obj_type'] is None:
                continue
            cell_info['wallidx_i_j'] = (wall_idxx, i, j)
            cell_info_grid_GT.append(cell_info)

            obj_random_id = cell_info['emitter_info']['random_id']
            emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


#             cell_intensity = np.array([emitter_prop_total3d['intensity_scale255'] * x * 255.for x in emitter_prop_total3d['intensity_scaled01']]) # intensity_scaled: [0., 1.]
            cell_intensity = np.array(emitter_prop_total3d['intensity']) * env_scale * hdr_scale
            cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
            cell_info['emitter_info']['intensity'] = [x * env_scale * hdr_scale for x in emitter_prop_total3d['intensity']]
            intensity_scale255 = max(cell_info['emitter_info']['intensity']) / 255.
            intensity_scaled01 = [np.clip(x / (intensity_scale255+1e-5) / 255., 0., 1.) for x in cell_info['emitter_info']['intensity']]
            cell_info['emitter_info']['intensity_scaled01'] = intensity_scaled01

            # other representation-specific params
            if cell_info['obj_type'] == 'window':
                cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
#                 print(cell_info['emitter_info']['light_dir_abs'], obj_random_id)
                if emitter_representation_type in ['1ambient']:
                    cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                if emitter_representation_type in ['2ambient']:
                    cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                    cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']



plt.figure(figsize=(15, 8))
ax_2d = plt.gca()

grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
import imageio
rgb_img = imageio.imread(sequence['rgb_img_path'])
gt_meshes = gt_obj_paths_list_normalized

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, gt_layout, gt_boxes_dict_RE, None, gt_meshes, gt_meshes, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'}, \
                pickle_id=pickle_id, if_hide_invalid_cats=True)

%matplotlib widget

img_map2 = scene_box.draw_projected_bdb3d('GT', return_plt=True, if_save=False, save_path='', if_use_plt=True, if_vis_2dbbox=True, 
                                          if_vis_2dbbox_iou=False, if_vis_3dbboxproj=True, if_vis_2dbbox_random_id=True)
img_map2.show()

# fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('both', if_show_objs=True, hide_random_id=False, if_print_log=True, hide_cells=True, if_dump_to_mesh=True)
fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('GT', if_debug=True, if_show_objs=True, hide_random_id=False, hide_cells=True, if_dump_to_mesh=True, if_show_emitter=False, fig_scale=2.)

mainDiffMat_xml/scene0403_01/im_12.hdr
/data/ruizhu/OR-V4full-withMesh20210506-val500-OR45_total3D_train_test_data/mainDiffMat_xml/scene0403_01/layout_obj_12_reindexed.pkl
dict_keys(['rgb_img_path', 'bins_tensor', 'est_data', 'gt_cam_R', 'cam_K', 'gt_layout', 'pre_layout', 'pre_cam_R', 'image'])
dict_keys(['cells_vis_info_list', 'cell_info_grid_GT', 'cell_info_grid_PRED', 'emitter_cls_prob_PRED', 'emitter_cls_prob_GT', 'pre_layout', 'pre_cam_R', 'envmap_lightAccu_mean_vis_GT'])
0.0 0.14788189987163028
dict_keys(['rgb_img_path', 'depth_map', 'boxes', 'camera', 'layout', 'scene_name', 'withinsequence_id', 'meta_split', 'meta_name', 'sub_name', 'scene_pickle_file', 'cam_pickle_file', 'frame_pickle_file', 'reindex_info_dict'])
dict_keys(['sequence_name', 'withinsequence_id', 'boxes', 'emitter_random_ids']) scene0403_01 12
dict_keys(['emitter2wall_assign_info_list', 'emitters_obj_list', 'wall_grid_prob', 'cell_prob_mean', 'cell_prob', 'cell_count', 'cell_info_grid'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 [Skipped] 41:door not valid of dataset: OR
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 [Skipped] 41:door not valid of dataset: OR
0 PGTQ2 table /home/ruizhu/Documents/data/OR-sampledMeshes/04379243/7fd9f6bc9a89d240800a0120f3e757a2/alignedNew_normalized.obj
1 480BF chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/4178fab56c04a32ae8f8c11a24c52ebb/alignedNew_normalized.obj
2 WJGJD chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/4178fab56c04a32ae8f8c11a24c52ebb/alignedNew_normalized.obj
3 B6QXK chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/2952ec189e993eeb17e114a7b3c6fb1/alignedNew_normalized.obj
4 ZZI40 chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/2952ec189e993eeb17e114a7b3c6fb1/alignedNew_normalized.obj
5 6GQKR chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/2952ec189e993eeb17e114a7b3c6fb1/alignedNew_normalized.obj
6 MEUZM chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/2952ec189e993eeb17e114a7b3c6fb1/alignedNew_normalized.obj
7 14PYZ chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/295

In [144]:
p = a['sampled_points']

In [148]:
with open('../tmp/2952ec189e993eeb17e114a7b3c6fb1/sampled.pickle', 'rb') as f:
     a= pickle.load(f)

%matplotlib widget

fig = plt.figure(figsize=(8, 8 ))

ax_3d_GT = fig.add_subplot(111, projection='3d')

ax_3d_GT.set_proj_type('ortho')

ax_3d_GT.set_aspect("auto")

ax_3d_GT.view_init(elev=-42, azim=111)

ax_3d_GT.scatter(p[:, 0], p[:, 1], p[:, 2], s=2)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
gt_boxes_dict_RE

{'if_valid': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 'coeffs': [array([1.1641, 0.3881, 0.874 ]),
  array([0.239 , 0.4215, 0.2603]),
  array([0.2396, 0.407 , 0.2812]),
  array([0.3202, 0.4481, 0.3588]),
  array([0.3216, 0.4461, 0.3221]),
  array([0.3283, 0.4539, 0.4058]),
  array([0.2908, 0.445 , 0.3354]),
  array([0.3005, 0.4321, 0.35  ]),
  array([0.3599, 0.4006, 0.3882]),
  array([0.468 , 1.1148, 0.1271]),
  array([0.4806, 1.1166, 0.2099])],
 'basis': [array([[ 0.9498,  0.    , -0.3129],
         [ 0.    ,  1.    ,  0.    ],
         [ 0.3129,  0.    ,  0.9498]]),
  array([[ 0.1186,  0.    ,  0.9929],
         [ 0.    ,  1.    ,  0.    ],
         [-0.9929,  0.    ,  0.1186]]),
  array([[ 0.9912,  0.    , -0.1325],
         [ 0.    ,  1.    ,  0.    ],
         [ 0.1325,  0.    ,  0.9912]]),
  array([[ 0.9576,  0.    , -0.2879],
         [ 0.    ,  1.    ,  0.    ],
         [ 0.2879,  0.    ,  0.9576]]),
  array([[ 0.9239,  0.    ,  0

In [139]:
for pickle_id in range(len(split)):

    naming = split[pickle_id][2]
    print(naming)

    # naming = 'main_xml/scene0096_00/im_6.hdr'
    # naming = 'main_xml1/scene0552_00/im_1.hdr'
    # naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
    # naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
    # pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
    meta_split = naming.split('/')[0]
    scene_name = naming.split('/')[1]
    withinsequence_id = int(naming.split('/')[2].split('.')[0].split('_')[1])
    # assert pickle_id==withinsequence_id

    train_test_data_path = '/data/ruizhu/OR-V4full-withMesh20210507-val500-OR45_total3D_train_test_data'

    pickle_path_ori = '%s/%s/%s/%s'%(train_test_data_path, naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


    grid_size = 8

    pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
    pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
    pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

    print(pickle_path)

    # ======= read saved results from evaluation

    # GT envmap input + overfit to val200
    # results_folder = '20210503-170713-train_POD_emitterEst_lightAccuV3_bs2x4_GTsampledEnvMap-GTBRDFLight_overFitVal_fixedDataScaleCellOrder0502data'; tid=10000

    # est
    results_folder = '20210503-155348-train_POD_emitterEst_lightAccuV3_bs2x4_RuitrainedBRDF-light_fixedDataScaleCellOrder0502data'; tid=12000

    layout_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_layout_info_tid%d-%d.pickle'%(results_folder, tid, 0)
    emitter_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_cells_vis_info_list_tid%d-%d.pickle'%(results_folder, tid, 0)
    results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%(results_folder, 0)

    with open(layout_results_pickle_path, 'rb') as f:
        layout_pickle_dict = pickle.load(f)
        print(layout_pickle_dict.keys())
    with open(emitter_results_pickle_path, 'rb') as f:
        emitter_pickle_dict = pickle.load(f)
        print(emitter_pickle_dict.keys())

    cells_vis_info_list = emitter_pickle_dict['cells_vis_info_list']
    cam_R_pred = layout_pickle_dict['pre_cam_R']
    pred_layout = layout_pickle_dict['pre_layout']
    emitter_cls_prob_PRED = emitter_pickle_dict['emitter_cls_prob_PRED']
    cell_info_grid_PRED = emitter_pickle_dict['cell_info_grid_PRED']


    with open(results_emitter_input_pickle_path, 'rb') as f:
        results_emitter_input_pickle_dict = pickle.load(f)

    env_scale, hdr_scale = results_emitter_input_pickle_dict['env_scale'], results_emitter_input_pickle_dict['hdr_scale']
    print(env_scale, hdr_scale)

    import pickle
    with open(pickle_path, 'rb') as f:
        sequence = pickle.load(f)
    print(sequence.keys())

    with open(pickle_emitters_path, 'rb') as f:
        sequence_emitters = pickle.load(f)
    print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
    with open(pickle_emitters_assign_info_list_path, 'rb') as f:
        sequence_emitter2wall_assign_info_dict = pickle.load(f)
    print(sequence_emitter2wall_assign_info_dict.keys())

    import pickle5 as pickle
    emitter_representation_type = '0ambient'
    emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
    with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
        emitters_prop_dict_representation_dict = pickle.load(f)

    # transform_to_total3d_coords_dict = {'transform_R': frame.transform_R, 'transform_t': frame.transform_t}
    transform_to_total3d_coords_dict_path = Path(train_test_data_path) / meta_split / scene_name / ('transform_to_total3d_coords_dict_%d.pkl'%withinsequence_id)
    import pickle5 as pickle
    with open(transform_to_total3d_coords_dict_path, 'rb') as f:
        transform_to_total3d_coords_dict = pickle.load(f)

    # RAW_data_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layout_labels_V4full-detachEmitterRERERE20210420/'
    # emitter_dict_pickle_file = Path(RAW_data_path) / meta_split / scene_name / ('emitter_dict_%s_%d.pickle'%('0ambient', withinsequence_id))
    # with open(emitter_dict_pickle_file, "rb") as f:
    #     emitter_dict_RAW = pickle.load(f)


    %matplotlib widget

    gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
    gt_boxes_dict_RE = {key: [x[key] for x in sequence['boxes']['bdb3D_full']] for key in sequence['boxes']['bdb3D_full'][0]}
    len_keys = [len(gt_boxes_dict_RE[key]) for key in gt_boxes_dict_RE]
    assert len(list(set(len_keys))) == 1
    gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
    random_id_list = [x['random_id'] for x in sequence['boxes']['bdb2D_full']]

    gt_obj_paths_list_normalized = [x.replace('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped', '/home/ruizhu/Documents/data/OR-sampledMeshes').replace('alignedNew.obj', 'alignedNew_normalized.obj') for x in gt_obj_paths_list]
    assert len(gt_obj_paths_list_normalized)==len(gt_obj_paths_list)==len_keys[0]
    gt_boxes_dict_RE['obj_path_normalized'] = gt_obj_paths_list_normalized
    gt_boxes_dict_RE['random_id'] = random_id_list
    gt_boxes_dict_RE['bdb2d'] = [[x['x1'], x['y1'], x['x2'], x['y2']] for x in sequence['boxes']['bdb2D_full']]
    gt_boxes_dict_RE['mask'] = [x for x in sequence['boxes']['mask']]
    assert len(gt_boxes_dict_RE['mask']) == len(gt_boxes_dict_RE['bdb2d'])
    assert len(gt_boxes_dict_RE['mask']) == len(sequence['boxes']['if_valid'])
    for bbox_idx in range(len(gt_boxes_dict_RE['if_valid'])):
        if gt_boxes_dict_RE['mask'][bbox_idx] is None:
            gt_boxes_dict_RE['if_valid'][bbox_idx] = False
    #     if sequence['boxes']['bdb2D_full']['if_is_indoor_obj'] == False:
    #         gt_boxes_dict_RE['if_valid'][bbox_idx] = False
    #     else:

    gt_layout = sequence['layout']['bdb3D']

    from utils.utils_total3D.data_config import Dataset_Config
    from utils.utils_total3D.utils_OR_layout import to_dict_tensor

    from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

    OR = 'OR45'
    dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
    bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
    cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                      torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                      torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

    # ======== emitters_list
    emitters_obj_list = []
    for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
        obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                        'random_id': sequence_emitters['boxes']['random_id'][x], \
                        'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                        'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                        'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                        'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
        obj_random_id = sequence_emitters['boxes']['random_id'][x]
        emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
        if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window' and 'light_center_world_total3d' in emitter_prop_total3d:
            light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
            light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
        else:
            light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
            light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
        obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]

        obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

        emitters_obj_list.append(obj_dict_new)

    emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
    cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
    cell_info_grid_GT = []
    for wall_idxx in range(6):
        for i in range(grid_size):
            for j in range(grid_size):
                cell_info = cell_info_grid_GT_includeempty[wall_idxx * grid_size**2 + i * grid_size + j]
                if cell_info['obj_type'] is None:
                    continue
                cell_info['wallidx_i_j'] = (wall_idxx, i, j)
                cell_info_grid_GT.append(cell_info)

                obj_random_id = cell_info['emitter_info']['random_id']
                emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


    #             cell_intensity = np.array([emitter_prop_total3d['intensity_scale255'] * x * 255.for x in emitter_prop_total3d['intensity_scaled01']]) # intensity_scaled: [0., 1.]
                cell_intensity = np.array(emitter_prop_total3d['intensity']) * env_scale * hdr_scale
                cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
                cell_info['emitter_info']['intensity'] = [x * env_scale * hdr_scale for x in emitter_prop_total3d['intensity']]
                intensity_scale255 = max(cell_info['emitter_info']['intensity']) / 255.
                intensity_scaled01 = [np.clip(x / (intensity_scale255+1e-5) / 255., 0., 1.) for x in cell_info['emitter_info']['intensity']]
                cell_info['emitter_info']['intensity_scaled01'] = intensity_scaled01

                # other representation-specific params
                if cell_info['obj_type'] == 'window':
                    cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                    cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
    #                 print(cell_info['emitter_info']['light_dir_abs'], obj_random_id)
                    if emitter_representation_type in ['1ambient']:
                        cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                    if emitter_representation_type in ['2ambient']:
                        cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                        cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']



    plt.figure(figsize=(15, 8))
    ax_2d = plt.gca()

    grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
    import imageio
    rgb_img = imageio.imread(sequence['rgb_img_path'])
    gt_meshes = gt_obj_paths_list_normalized

    scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, gt_layout, gt_boxes_dict_RE, None, gt_meshes, gt_meshes, 'GT', None, dataset=dataset, OR='OR45', \
                    emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                    emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                    grid_size=grid_size, \
                    paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'}, \
                    pickle_id=pickle_id, if_hide_invalid_cats=True)

    %matplotlib widget

    img_map2 = scene_box.draw_projected_bdb3d('GT', return_plt=True, if_save=False, save_path='', if_use_plt=True, if_vis_2dbbox=True, 
                                              if_vis_2dbbox_iou=False, if_vis_3dbboxproj=True, if_vis_2dbbox_random_id=True)
    img_map2.show()

    # fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('both', if_show_objs=True, hide_random_id=False, if_print_log=True, hide_cells=True, if_dump_to_mesh=True)
    fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('GT', if_show_objs=True, hide_random_id=False, if_debug=True, hide_cells=True, if_dump_to_mesh=True, if_show_emitter=False, fig_scale=2.)

mainDiffLight_xml1/scene0509_00/im_1.hdr
/data/ruizhu/OR-V4full-withMesh20210507-val500-OR45_total3D_train_test_data/mainDiffLight_xml1/scene0509_00/layout_obj_1_reindexed.pkl
dict_keys(['rgb_img_path', 'bins_tensor', 'est_data', 'gt_cam_R', 'cam_K', 'gt_layout', 'pre_layout', 'pre_cam_R', 'image'])
dict_keys(['cells_vis_info_list', 'cell_info_grid_GT', 'cell_info_grid_PRED', 'emitter_cls_prob_PRED', 'emitter_cls_prob_GT', 'pre_layout', 'pre_cam_R', 'envmap_lightAccu_mean_vis_GT'])
0.0 0.14788189987163028
dict_keys(['rgb_img_path', 'depth_map', 'boxes', 'camera', 'layout', 'scene_name', 'withinsequence_id', 'meta_split', 'meta_name', 'sub_name', 'scene_pickle_file', 'cam_pickle_file', 'frame_pickle_file', 'reindex_info_dict'])
dict_keys(['sequence_name', 'withinsequence_id', 'boxes', 'emitter_random_ids']) scene0509_00 1
dict_keys(['emitter2wall_assign_info_list', 'emitters_obj_list', 'wall_grid_prob', 'cell_prob_mean', 'cell_prob', 'cell_count', 'cell_info_grid'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 TXZKQ file_cabinet /home/ruizhu/Documents/data/OR-sampledMeshes/03337140/abe41b86df0dcd5a3da04ac9f7df60de/alignedNew_normalized.obj
1 LLGM5 trash_bin /home/ruizhu/Documents/data/OR-sampledMeshes/02747177/93d5f21e94ac5fb68a606c63493ddb7c/alignedNew_normalized.obj
2 LK0QD chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/4f4b6167ef11ae4e8f8c11a24c52ebb/alignedNew_normalized.obj
3 99EUT cabinet /home/ruizhu/Documents/data/OR-sampledMeshes/02933112/4ec97e83a1e9725c77a332e31ab74b68/alignedNew_normalized.obj
4 STWLJ monitor /home/ruizhu/Documents/data/OR-sampledMeshes/03211117/bbfac23f57b5fbd820e867fd9fb7164/alignedNew_normalized.obj
5 6LLJC table /home/ruizhu/Documents/data/OR-sampledMeshes/04379243/62c6addae5defe8bc62ff677c806df30/alignedNew_normalized.obj
6 8BIUY chair /home/ruizhu/Documents/data/OR-sampledMeshes/03001627/9368cd9028151e1e9d51a07a5989d077/alignedNew_normalized.obj
[GT] Mesh written to scene_mesh_debug_val-499.obj
main_xml1/scene0272_01/im_6.hdr
/data/ruizhu/OR-

FileNotFoundError: [Errno 2] No such file or directory: '/data/ruizhu/OR-V4full-withMesh20210507-val500-OR45_total3D_train_test_data/main_xml1/scene0272_01/layout_obj_6_reindexed.pkl'

In [70]:
sequence['boxes']['bdb2D_full']

array([{'if_valid': False, 'class_id': 0, 'cat_name': 'None', 'x1': 0.0, 'y1': 0.0, 'x2': 1.0, 'y2': 1.0, 'obj_path': '', 'if_is_object': 'None', 'if_is_indoor_obj': 'None', 'vis_ratio': -1.0}],
      dtype=object)

In [27]:
gt_boxes_dict_RE.keys()

dict_keys(['if_valid', 'coeffs', 'basis', 'centroid', 'class_id', 'cat_name', 'random_id', 'axes_RAW_order', 'obj_path_normalized', 'bdb2d'])

In [98]:
# mask_all = np.zeros((480, 640))
mask_all = np.zeros((240, 320))
obj_idx = 1
for x_idx, x in enumerate(sequence['boxes']['mask']):
#     plt.figure()
    if x is None:
        continue
#     mask = x['msk'].astype(np.float32)

    x1, y1, x2, y2 = x['msk_bdb_half']
    dest_mask = mask_all[y1:y2+1, x1:x2+1]==0
    ori_mask = x['msk_half'].astype(np.uint8)
    ori_mask[ori_mask==1] += obj_idx
    mask_all[y1:y2+1, x1:x2+1][dest_mask] = ori_mask[dest_mask]
    print(obj_idx)
    obj_idx += 1
    
    
    %matplotlib widget

#     plt.figure()
#     plt.imshow(ori_mask)
#     plt.title(x_idx)
#     plt.show()

%matplotlib widget

plt.figure()
plt.imshow(mask_all)
plt.title('Combined')
plt.show()

1
2
3
4
5
6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# utils

In [99]:
import os.path as osp
import struct
from PIL import Image

def loadBinary(imName, channels = 1, dtype=np.float32, if_resize=True):
    assert dtype in [np.float32, np.int32], 'Invalid binary type outside (np.float32, np.int32)!'
    if not(osp.isfile(imName ) ):
        assert(False ), '%s doesnt exist!'%imName
    with open(imName, 'rb') as fIn:
        hBuffer = fIn.read(4)
        height = struct.unpack('i', hBuffer)[0]
        wBuffer = fIn.read(4)
        width = struct.unpack('i', wBuffer)[0]
        dBuffer = fIn.read(4 * channels * width * height )
        if dtype == np.float32:
            decode_char = 'f'
        elif dtype == np.int32:
            decode_char = 'i'
        depth = np.asarray(struct.unpack(decode_char * channels * height * width, dBuffer), dtype=dtype)
        depth = depth.reshape([height, width, channels] )

        depth = np.squeeze(depth)

    return depth[np.newaxis, :, :]

def loadImage(imName, isGama = False):
    if not(osp.isfile(imName ) ):
        assert(False), 'File does not exist: ' + imName 

    im = Image.open(imName)

    im = np.asarray(im, dtype=np.float32)
    if isGama:
        im = (im / 255.0) ** 2.2
        im = 2 * im - 1
    else:
        im = (im - 127.5) / 127.5
    if len(im.shape) == 2:
        im = im[:, np.newaxis]
    im = np.transpose(im, [2, 0, 1] )

    return im

